In [1]:
import pandas as pd
import numpy as np

import scipy.stats as stats
from statsmodels.stats.multitest import multipletests

import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
data = {
    'Model': ['YOLOv5n']*10 + ['YOLOv6n']*10 + ['YOLOv8n']*10 + ['YOLOv9t']*10 + ['YOLOv10n']*10 + ['YOLOv11n']*10 + ['YOLOv12n']*10,
    'Angle': [-25, -20, -15, -10, -5, 5, 10, 15, 20, 25]*7,
    'Person_Precision': [
        # YOLOv5n
        0.535, 0.571, 0.539, 0.615, 0.779, 0.779, 0.705, 0.67, 0.479, 0.573,
        # YOLOv6n
        0.541, 0.608, 0.656, 0.717, 0.759, 0.776, 0.744, 0.668, 0.608, 0.563,
        # YOLOv8n
        0.529, 0.543, 0.625, 0.694, 0.776, 0.758, 0.753, 0.608, 0.588, 0.541,
        # YOLOv9t
        0.542, 0.445, 0.584, 0.697, 0.756, 0.737, 0.676, 0.671, 0.608, 0.564,
        # YOLOv10n
        0.478, 0.535, 0.458, 0.585, 0.666, 0.693, 0.625, 0.506, 0.582, 0.514,
        # YOLOv11n
        0.533, 0.608, 0.541, 0.641, 0.739, 0.768, 0.713, 0.667, 0.537, 0.435,
        # YOLOv12n
        0.517, 0.562, 0.57, 0.637, 0.712, 0.721, 0.638, 0.498, 0.521, 0.491
    ],
    'Person_Recall': [
        # YOLOv5n
        0.46, 0.503, 0.613, 0.646, 0.609, 0.609, 0.567, 0.476, 0.545, 0.397,
        # YOLOv6n
        0.445, 0.483, 0.528, 0.573, 0.616, 0.591, 0.518, 0.486, 0.443, 0.396,
        # YOLOv8n
        0.447, 0.551, 0.574, 0.61, 0.628, 0.657, 0.538, 0.561, 0.474, 0.429,
        # YOLOv9t
        0.431, 0.605, 0.591, 0.582, 0.621, 0.655, 0.605, 0.528, 0.473, 0.43,
        # YOLOv10n
        0.473, 0.499, 0.622, 0.607, 0.598, 0.598, 0.583, 0.578, 0.467, 0.423,
        # YOLOv11n
        0.385, 0.416, 0.554, 0.583, 0.597, 0.558, 0.509, 0.436, 0.481, 0.507,
        # YOLOv12n
        0.433, 0.474, 0.531, 0.562, 0.575, 0.558, 0.515, 0.561, 0.444, 0.403
    ],
    'Person_mAP': [
        # YOLOv5n
        0.435, 0.496, 0.57, 0.638, 0.709, 0.709, 0.629, 0.539, 0.471, 0.426,
        # YOLOv6n
        0.417, 0.49, 0.544, 0.623, 0.683, 0.675, 0.597, 0.522, 0.451, 0.402,
        # YOLOv8n
        0.427, 0.501, 0.557, 0.639, 0.711, 0.723, 0.63, 0.537, 0.467, 0.416,
        # YOLOv9t
        0.416, 0.493, 0.555, 0.636, 0.692, 0.712, 0.633, 0.555, 0.486, 0.424,
        # YOLOv10n
        0.4, 0.472, 0.534, 0.596, 0.665, 0.668, 0.601, 0.518, 0.467, 0.393,
        # YOLOv11n
        0.401, 0.459, 0.514, 0.603, 0.67, 0.669, 0.591, 0.517, 0.457, 0.401,
        # YOLOv12n
        0.403, 0.47, 0.514, 0.581, 0.642, 0.638, 0.561, 0.488, 0.425, 0.371
    ],
    'Car_Precision': [
        # YOLOv5n
        0.589, 0.641, 0.637, 0.725, 0.843, 0.817, 0.764, 0.691, 0.514, 0.546,
        # YOLOv6n
        0.568, 0.633, 0.695, 0.788, 0.807, 0.837, 0.79, 0.643, 0.544, 0.499,
        # YOLOv8n
        0.589, 0.598, 0.698, 0.783, 0.845, 0.82, 0.831, 0.641, 0.6, 0.54,
        # YOLOv9t
        0.618, 0.532, 0.662, 0.785, 0.849, 0.839, 0.782, 0.686, 0.604, 0.525,
        # YOLOv10n
        0.481, 0.515, 0.485, 0.619, 0.703, 0.701, 0.635, 0.492, 0.513, 0.438,
        # YOLOv11n
        0.53, 0.595, 0.52, 0.643, 0.722, 0.743, 0.693, 0.616, 0.446, 0.352,
        # YOLOv12n
        0.56, 0.611, 0.628, 0.717, 0.775, 0.774, 0.726, 0.535, 0.545, 0.515
    ],
    'Car_Recall': [
        # YOLOv5n
        0.493, 0.564, 0.691, 0.753, 0.741, 0.762, 0.705, 0.579, 0.568, 0.455,
        # YOLOv6n
        0.489, 0.525, 0.587, 0.669, 0.736, 0.73, 0.66, 0.547, 0.477, 0.446,
        # YOLOv8n
        0.526, 0.591, 0.659, 0.735, 0.767, 0.782, 0.688, 0.61, 0.514, 0.47,
        # YOLOv9t
        0.496, 0.619, 0.658, 0.723, 0.763, 0.788, 0.739, 0.584, 0.512, 0.454,
        # YOLOv10n
        0.456, 0.503, 0.639, 0.695, 0.717, 0.729, 0.688, 0.597, 0.483, 0.418,
        # YOLOv11n
        0.534, 0.591, 0.706, 0.792, 0.812, 0.799, 0.755, 0.624, 0.583, 0.564,
        # YOLOv12n
        0.499, 0.565, 0.651, 0.72, 0.734, 0.744, 0.716, 0.637, 0.521, 0.473
    ],
    'Car_mAP': [
        # YOLOv5n
        0.473, 0.547, 0.658, 0.774, 0.829, 0.84, 0.769, 0.582, 0.459, 0.403,
        # YOLOv6n
        0.438, 0.507, 0.599, 0.743, 0.814, 0.818, 0.736, 0.531, 0.404, 0.359,
        # YOLOv8n
        0.489, 0.553, 0.66, 0.785, 0.856, 0.853, 0.783, 0.599, 0.484, 0.418,
        # YOLOv9t
        0.47, 0.525, 0.646, 0.769, 0.833, 0.854, 0.781, 0.589, 0.49, 0.407,
        # YOLOv10n
        0.385, 0.441, 0.548, 0.692, 0.772, 0.777, 0.694, 0.519, 0.406, 0.351,
        # YOLOv11n
        0.466, 0.547, 0.632, 0.782, 0.835, 0.839, 0.765, 0.569, 0.458, 0.398,
        # YOLOv12n
        0.456, 0.532, 0.632, 0.749, 0.8, 0.802, 0.755, 0.576, 0.465, 0.412
    ],
    'Bicycle_Precision': [
        # YOLOv5n
        0.207, 0.286, 0.322, 0.432, 0.62, 0.608, 0.528, 0.489, 0.245, 0.267,
        # YOLOv6n
        0.195, 0.327, 0.439, 0.584, 0.619, 0.664, 0.631, 0.491, 0.373, 0.284,
        # YOLOv8n
        0.221, 0.265, 0.44, 0.554, 0.67, 0.63, 0.686, 0.483, 0.402, 0.325,
        # YOLOv9t
        0.27, 0.234, 0.398, 0.573, 0.668, 0.658, 0.598, 0.582, 0.471, 0.361,
        # YOLOv10n
        0.188, 0.286, 0.283, 0.458, 0.617, 0.639, 0.539, 0.384, 0.405, 0.307,
        # YOLOv11n
        0.209, 0.309, 0.3, 0.41, 0.543, 0.563, 0.496, 0.446, 0.274, 0.172,
        # YOLOv12n
        0.244, 0.333, 0.436, 0.53, 0.645, 0.653, 0.58, 0.374, 0.365, 0.308
    ],
    'Bicycle_Recall': [
        # YOLOv5n
        0.285, 0.349, 0.544, 0.625, 0.61, 0.653, 0.609, 0.525, 0.521, 0.341,
        # YOLOv6n
        0.256, 0.326, 0.403, 0.48, 0.535, 0.534, 0.506, 0.475, 0.448, 0.398,
        # YOLOv8n
        0.305, 0.411, 0.505, 0.555, 0.597, 0.644, 0.527, 0.549, 0.429, 0.378,
        # YOLOv9t
        0.313, 0.526, 0.536, 0.541, 0.586, 0.618, 0.61, 0.541, 0.49, 0.402,
        # YOLOv10n
        0.221, 0.274, 0.484, 0.492, 0.492, 0.508, 0.502, 0.519, 0.371, 0.327,
        # YOLOv11n
        0.287, 0.338, 0.527, 0.576, 0.597, 0.594, 0.547, 0.477, 0.512, 0.497,
        # YOLOv12n
        0.2, 0.257, 0.366, 0.385, 0.387, 0.455, 0.443, 0.49, 0.374, 0.296
    ],
    'Bicycle_mAP': [
        # YOLOv5n
        0.153, 0.236, 0.405, 0.567, 0.654, 0.672, 0.599, 0.462, 0.305, 0.223,
        # YOLOv6n
        0.155, 0.264, 0.395, 0.525, 0.601, 0.625, 0.568, 0.463, 0.353, 0.254,
        # YOLOv8n
        0.172, 0.239, 0.439, 0.578, 0.674, 0.682, 0.625, 0.49, 0.366, 0.277,
        # YOLOv9t
        0.219, 0.321, 0.454, 0.571, 0.655, 0.685, 0.645, 0.536, 0.418, 0.288,
        # YOLOv10n
        0.11, 0.192, 0.347, 0.467, 0.549, 0.586, 0.507, 0.406, 0.277, 0.19,
        # YOLOv11n
        0.159, 0.233, 0.398, 0.519, 0.594, 0.605, 0.522, 0.431, 0.33, 0.236,
        # YOLOv12n
        0.14, 0.213, 0.346, 0.431, 0.505, 0.529, 0.475, 0.377, 0.27, 0.189
    ]
}

# Создаем DataFrame и преобразуем в длинный формат
df = pd.DataFrame(data)
df_long = pd.melt(df, id_vars=['Model', 'Angle'], 
                 var_name='Metric_Class', value_name='Value')
df_long[['Class', 'Metric']] = df_long['Metric_Class'].str.split('_', expand=True)
df_long = df_long.drop(columns=['Metric_Class'])

In [5]:
def stat_diff_check(list_1, list_2, list1_name='list1', list2_name='list2', alpha=0.05):
    """
    Усовершенствованная функция для сравнения двух зависимых выборок.
    
    Параметры:
    - list_1, list_2: сравниваемые выборки
    - list1_name, list2_name: названия выборок для вывода
    - alpha: уровень значимости (по умолчанию 0.05)
    """
    # Проверка на одинаковую длину выборок
    if len(list_1) != len(list_2):
        raise ValueError("Выборки должны быть одинаковой длины для парного сравнения")
    
    # Проверка нормальности исходных выборок и их разностей
    _, p1 = stats.shapiro(list_1)
    _, p2 = stats.shapiro(list_2)
    diff = np.array(list_1) - np.array(list_2)
    _, p_diff = stats.shapiro(diff)
    
    # Вывод информации о нормальности
    print("\n" + "="*50)
    print(f"Статистический анализ: {list1_name} vs {list2_name}")
    print("="*50)
    print(f"Нормальность {list1_name}: {'Да' if p1 > alpha else 'Нет'} (p={p1:.3f})")
    print(f"Нормальность {list2_name}: {'Да' if p2 > alpha else 'Нет'} (p={p2:.3f})")
    print(f"Нормальность разностей: {'Да' if p_diff > alpha else 'Нет'} (p={p_diff:.3f})")
    
    # Выбор теста в зависимости от нормальности разностей
    if p_diff > alpha:  # Если разности нормальны
        t_stat, p_value = stats.ttest_rel(list_1, list_2)
        test_name = "Парный t-тест"
        # Расчет Cohen's d для парных выборок
        cohen_d = np.mean(diff) / np.std(diff, ddof=1)
        effect_size = f"Cohen's d = {cohen_d:.2f}"
    else:
        w_stat, p_value = stats.wilcoxon(list_1, list_2)
        test_name = "Тест Уилкоксона"
        # Расчет рангового коэффициента корреляции как меры эффекта
        r = w_stat / (np.sqrt(len(list_1) * (2*len(list_1) + 1)/6))
        effect_size = f"Ранговый коэффициент r = {r:.2f}"
    
    # Вывод результатов теста
    print(f"\nРезультаты ({test_name}):")
    print(f"Среднее {list1_name}: {np.mean(list_1):.3f} ± {np.std(list_1, ddof=1):.3f}")
    print(f"Среднее {list2_name}: {np.mean(list_2):.3f} ± {np.std(list_2, ddof=1):.3f}")
    print(f"p-value = {p_value:.4f}")
    # print(f"Размер эффекта: {effect_size}")
    
    # Интерпретация результатов
    print("\nВывод:")
    if p_value < alpha:
        print(f"Различия статистически значимы (p < {alpha})")
        if np.mean(list_1) > np.mean(list_2):
            print(f"Значения в {list1_name} значимо выше")
        else:
            print(f"Значения в {list2_name} значимо выше")
    else:
        print(f"Различия не статистически значимы (p ≥ {alpha})")
        print("~"*50)
        print("~"*50)
    
    # Визуализация
    # plt.figure(figsize=(10, 6))
    # sns.boxplot(data=[list_1, list_2], palette="Set2", width=0.5)
    # sns.stripplot(data=[list_1, list_2], color="black", alpha=0.5, jitter=True)
    # plt.xticks([0, 1], [list1_name, list2_name])
    # plt.ylabel('Значения метрик')
    # plt.title(f'Сравнение распределений: {list1_name} vs {list2_name}')
    # plt.grid(True, linestyle='--', alpha=0.3)
    # plt.show()

In [7]:
# Получаем уникальные модели
models = df_long['Model'].unique()

# Пары углов для сравнения (отрицательный vs положительный)
angle_pairs = [(-25, 25), (-20, 20), (-15, 15), (-10, 10), (-5, 5)]

# Проходим по каждой модели
for model in models:
    print(f"\n=== Модель: {model} ===")
    
    # Проходим по каждой паре углов
    for angle_neg, angle_pos in angle_pairs:
        print(f"\nСравнение углов: {angle_neg}° vs {angle_pos}°")
        
        # Фильтруем данные для отрицательного и положительного углов
        data_neg = df_long.query(
            'Model == @model and Angle == @angle_neg'
        )['Value'].tolist()
        
        data_pos = df_long.query(
            'Model == @model and Angle == @angle_pos'
        )['Value'].tolist()
        
        # Проверяем, что данные не пустые
        if not data_neg or not data_pos:
            print("Нет данных для сравнения.")
            continue
        
        # Вызываем функцию сравнения
        stat_diff_check(data_neg, data_pos)


=== Модель: YOLOv5n ===

Сравнение углов: -25° vs 25°

Статистический анализ: list1 vs list2
Нормальность list1: Да (p=0.272)
Нормальность list2: Да (p=0.848)
Нормальность разностей: Да (p=0.151)

Результаты (Парный t-тест):
Среднее list1: 0.403 ± 0.152
Среднее list2: 0.403 ± 0.116
p-value = 0.9954

Вывод:
Различия не статистически значимы (p ≥ 0.05)
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Сравнение углов: -20° vs 20°

Статистический анализ: list1 vs list2
Нормальность list1: Да (p=0.250)
Нормальность list2: Да (p=0.059)
Нормальность разностей: Да (p=0.729)

Результаты (Парный t-тест):
Среднее list1: 0.466 ± 0.141
Среднее list2: 0.456 ± 0.110
p-value = 0.7670

Вывод:
Различия не статистически значимы (p ≥ 0.05)
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Сравнение углов: -15° vs 15°

Статистический анализ: list1 vs list2
Нормальность list1: Да (p=0.299)
Нормальност